In [2]:
import pandas as pd
import numpy as np

제주관광공사 데이터

In [3]:
# 숙박업,관광지,음식점 모두 있음
Data1=pd.read_excel('../Database/제주관광공사_제주관광정보시스템(VISIT JEJU)_콘텐츠_20230315.xlsx')

문화빅데이터 데이터

In [4]:
#숙박업 2022/10
Data2=pd.read_csv('../Database/HW_LDGS_DAIL_MAX_AVRG_MIN_PRC_INFO_202212-10.csv')
#숙박업 2022/03
Data2_1=pd.read_csv('../Database/HW_LDGS_DAIL_MAX_AVRG_MIN_PRC_INFO_202303-11.csv')

제주데이터허브 데이터

In [6]:
#음식점 평점
Data3=pd.read_csv('../Database/NXE_3rd_ORIGIN_NXEGGRESTRTAL_20210101000000.csv')

In [7]:
#숙박업 평점
Data4=pd.read_csv('../Database/NXE_3rd_ORIGIN_NXEGGSTAYNGAL_20210101000000.csv')

In [8]:
#관광지 평점
Data5=pd.read_csv('../Database/NXE_3rd_ORIGIN_NXEGGTRRSRTAL_20210101000000.csv')

# 데이터를 정리해보자

## 숙박(실패...)

In [9]:
data1=pd.DataFrame(Data1)
data1["콘텐츠분류"].unique()

array(['숙박', '음식점', '관광지', '테마여행', '축제/행사', '쇼핑', '기타'], dtype=object)

In [43]:
data2=pd.DataFrame(Data2)
cols=data2.columns
data2_1=pd.DataFrame(Data2_1)

In [44]:
data2=data2.drop(columns=[cols[1],cols[5],cols[6]]).rename(columns={cols[0]:'이름',cols[2]:"지역1",cols[3]:"지역2",cols[4]:"주소"})
data2_1=data2_1.drop(columns=[cols[1],cols[5],cols[6]]).rename(columns={cols[0]:'이름',cols[2]:"지역1",cols[3]:"지역2",cols[4]:"주소"})

In [60]:
data2=data2[data2["지역1"]=="제주"]
data2_1=data2_1[data2_1["지역1"]=="제주"]

In [64]:
step1=pd.concat([data2,data2_1])

In [ ]:
step1.groupby("이름")['MIN_PRC'].apply(np.min)

In [ ]:
step1.groupby("이름")['AVRG_PRC'].apply(np.mean)

## 데이터 preprocessing

In [110]:
data3=pd.DataFrame(Data3)
data4=pd.DataFrame(Data4)
data5=pd.DataFrame(Data5)

In [129]:
add=[]
for x in [data3,data4,data5]:
    add.append(list(x.columns)) 

In [141]:
add=list(map(lambda x: dict(zip(["이름","구분","평점","day1"],x)),add))

In [123]:
result=[]
for x in [data3,data4,data5]:
    result.append(x.rename(columns=dict(zip(x.columns,["이름","구분","평점","구분1","day1","day2"]))).drop(columns=["구분1","day2"]))

In [154]:
for i in range(3):
    result[i].loc[len(result[2].index+1)]=add[i]

In [167]:
for i in range(3):
    result[i]["평점"]=result[i]["평점"].astype("float")

## 숙박

In [185]:
test1=pd.DataFrame(result[1].groupby("이름")["평점"].mean())
test1.index.name="기준"
test2=data1[data1["콘텐츠분류"]=="숙박"].set_index("제목")
test2.index.name="기준"
test3=pd.merge(test2,test1,how="left")

In [198]:
test3.to_excel('../수정데이터/숙박(평점포함).xlsx')

## 음식점

In [200]:
test1=pd.DataFrame(result[0].groupby("이름")["평점"].mean())
test1.index.name="기준"

In [203]:
test2=data1[data1["콘텐츠분류"]=="음식점"].set_index("제목")
test2.index.name="기준"

In [205]:
test3=pd.merge(test2,test1,how="left")

In [206]:
test3.to_excel('../수정데이터/음식점(평점포함).xlsx')

## 관광지

In [207]:
test1=pd.DataFrame(result[2].groupby("이름")["평점"].mean())
test1.index.name="기준"

In [210]:
test2=data1[data1["콘텐츠분류"]=="관광지"].set_index("제목")
test2.index.name="기준"

In [212]:
test3=pd.merge(test2,test1,how="left")

In [213]:
test3.to_excel('../수정데이터/관광지(평점포함).xlsx')

### 마지막 전처리

In [ ]:
data=pd.DataFrame(A)
drop_cols=[data.columns[i] for i in sum([list(range(26,42)),
                                         list(range(51,73))],[])]
drop_cols.extend(sum([
["웹사용여부","모바일사용여부","작성상태",'사용여부','공개여부','트립어드바이저코드','템플릿여부'],
["콘텐츠아이디","연관콘텐츠","외부검색어",'번역요청일시'],
["언어","작성상태_1","수정자","수정일시","우편번호"],
["웹사용여부","모바일사용여부","작성상태",'사용여부','공개여부','트립어드바이저코드','템플릿여부']],[]))
Use_data=data.drop(columns=drop_cols)

In [214]:
result={}
for name in ["숙박","음식점","관광지"]:
    data=pd.read_excel(f"../수정데이터/{name}(평점포함).xlsx")
    drop_cols=[data.columns[i] for i in sum([list(range(26,42)),
                                         list(range(51,73))],[])]
    drop_cols.extend(sum([
    ["웹사용여부","모바일사용여부","작성상태",'사용여부','공개여부','트립어드바이저코드','템플릿여부'],
    ["콘텐츠아이디","연관콘텐츠","외부검색어",'번역요청일시'],
    ["언어","작성상태_1","수정자","수정일시","우편번호"],
    ["웹사용여부","모바일사용여부","작성상태",'사용여부','공개여부','트립어드바이저코드','템플릿여부']],[]))
    Use_data=data.drop(columns=drop_cols)
    Use_data=Use_data[(Use_data["지번주소"].isna()==False)&(Use_data["도로명주소"].isna()==False)]
    result[name]=Use_data[Use_data["콘텐츠분류"]==name]

In [216]:
for name in result:
    result[name].to_excel(f"../수정데이터/{name}(최종).xlsx")